In [1]:
from copy import deepcopy
# import logging
import os

from pprint import PrettyPrinter
import xml.etree.ElementTree as ET
import xmltodict

from resources.ammos import AMMOS_CONVERSION
from resources.weapons import WEAPONS_CONVERSION

VANILLA_PATH = './vanilla/root'
PATH_OUTPUTS = './outputs'
DELETE_TOKEN = '!DELETE'

def validate_structure(input:ET, other:ET) -> bool:
    """Recursively compares the structure of two XML elements.
    
    Args:
    + input: First XML tree root.
    + other: Second XML tree root.
    
    Return: True if the structures are the same, False otherwise.
    """
    # Compare tags
    if input.tag != other.tag:
        return False

    # Compare the number of children
    if len(input) != len(other):
        return False

    # Recursively compare child elements
    for child_1, child_2 in zip(input, other):
        if not validate_structure(child_1, child_2):
            return False

    return True


pp = PrettyPrinter(indent=1, compact=True, sort_dicts=False)
def pprint(obj):
    return pp.pprint(obj)

In [2]:
# if os.path.exists('./log.log'):
#     os.remove('./log.log')

# # Create a logger
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)
# # Create handler (console and file)
# log_handler_console = logging.StreamHandler()
# log_handler_file = logging.FileHandler("log.log")
# log_handler_console.setLevel(logging.DEBUG)
# log_handler_file.setLevel(logging.DEBUG)
# formatter = logging.Formatter(
#     '%(asctime)s - %(name)s - %(levelname)s - %(message)s', 
#     datefmt='%Y-%m-%d %H:%M:%S',
# )
# log_handler_console.setFormatter(formatter)
# log_handler_file.setFormatter(formatter)
# # Add handlers to the logger
# logger.addHandler(log_handler_console)
# logger.addHandler(log_handler_file)

In [3]:
# Crawl list of all DLCs from latest dlclist.xml.
# Devlog: 2024-01-16, GTA V 1.70, have scanned up to patch2024_02
dlclist_xml = ET.parse(source=f'{VANILLA_PATH}/update/update.rpf/dlclist.xml').getroot()
dlclist = {'base': {'path': f'update/update.rpf'}}
for i in dlclist_xml.find('Paths'):
    dlc_archive, dlc_subpath = i.text.split(sep=':')
    
    if dlc_subpath[-1] == '/':
        dlc_subpath = dlc_subpath[0:-1]
    dlc = dlc_subpath.split(sep='/')[-1].lower()

    if dlc_archive == 'platform':
        dlc_path = f'x64w.rpf/dlcpacks/{dlc}'
    elif dlc_archive == 'dlcpacks':
        dlc_path = f'update/x64/dlcpacks/{dlc}/dlc.rpf'
    else:
        raise ValueError(f'Unknown archive type.')
    
    dlclist[dlc] = {'path': dlc_path}
    
for k_d, v_d in dlclist.items():
    print(f"{k_d.ljust(17)}: {v_d['path']}")

base             : update/update.rpf
mpbeach          : x64w.rpf/dlcpacks/mpbeach
mpbusiness       : x64w.rpf/dlcpacks/mpbusiness
mpchristmas      : x64w.rpf/dlcpacks/mpchristmas
mpvalentines     : x64w.rpf/dlcpacks/mpvalentines
mpbusiness2      : x64w.rpf/dlcpacks/mpbusiness2
mphipster        : x64w.rpf/dlcpacks/mphipster
mpindependence   : x64w.rpf/dlcpacks/mpindependence
mppilot          : x64w.rpf/dlcpacks/mppilot
spupgrade        : x64w.rpf/dlcpacks/spupgrade
mplts            : x64w.rpf/dlcpacks/mplts
mpheist          : update/x64/dlcpacks/mpheist/dlc.rpf
mppatchesng      : update/x64/dlcpacks/mppatchesng/dlc.rpf
patchday1ng      : update/x64/dlcpacks/patchday1ng/dlc.rpf
patchday2ng      : update/x64/dlcpacks/patchday2ng/dlc.rpf
mpchristmas2     : update/x64/dlcpacks/mpchristmas2/dlc.rpf
patchday2bng     : update/x64/dlcpacks/patchday2bng/dlc.rpf
patchday3ng      : update/x64/dlcpacks/patchday3ng/dlc.rpf
patchday4ng      : update/x64/dlcpacks/patchday4ng/dlc.rpf
mpluxe           :

In [4]:
# Scan for all weapon<...>.meta and pickups.meta files in all dlc archives.
for k_d, v_d in dlclist.items():
    dlclist[k_d].update({
        'base': {'weapons': {'avail': False, 'files': []}, 'pickups': {'avail': False}},
        'patch': {'weapons': {'avail': False, 'files': []}, 'pickups': {'avail': False}},
    })
    
    # Scan weapons in base update archive
    path_base_w = f"{VANILLA_PATH}/{v_d['path']}/common/data/ai"
    if os.path.isdir(path_base_w):
        for f in os.listdir(path_base_w):
            if ('weapon' in f) and (f.endswith('.meta')) :
                dlclist[k_d]['base']['weapons']['avail'] = True
                dlclist[k_d]['base']['weapons']['files'].append(f)
    # Scan weapons in DLC patch
    path_patch_w = f'{VANILLA_PATH}/update/update.rpf/dlc_patch/{k_d}/common/data/ai'
    if os.path.isdir(path_patch_w):
        for f in os.listdir(path_patch_w):
            if ('weapon' in f) and (f.endswith('.meta')) :
                dlclist[k_d]['patch']['weapons']['avail'] = True
                dlclist[k_d]['patch']['weapons']['files'].append(f)
        
    dlclist[k_d]['base']['pickups']['avail'] = os.path.isfile(f'{VANILLA_PATH}/{v_d["path"]}/common/data/pickups.meta')
    dlclist[k_d]['patch']['pickups']['avail'] = os.path.isfile(f'{VANILLA_PATH}/update/update.rpf/dlc_patch/{k_d}/common/data/pickups.meta')
    
for k_d in list(dlclist.keys()):
    if ((dlclist[k_d]['base']['pickups']['avail'] == False) and
        (dlclist[k_d]['base']['weapons']['avail'] == False) and
        (dlclist[k_d]['patch']['pickups']['avail'] == False) and
        (dlclist[k_d]['patch']['weapons']['avail'] == False)):
        dlclist.pop(k_d)

for k_d, v_d in dlclist.items():
    print(f"{k_d}: {v_d['base']}")
    print(f"{k_d}: {v_d['patch']}")

base: {'weapons': {'avail': True, 'files': ['weapons.meta']}, 'pickups': {'avail': True}}
base: {'weapons': {'avail': False, 'files': []}, 'pickups': {'avail': False}}
mpbeach: {'weapons': {'avail': False, 'files': []}, 'pickups': {'avail': False}}
mpbeach: {'weapons': {'avail': True, 'files': ['weaponsnspistol.meta']}, 'pickups': {'avail': True}}
mpbusiness: {'weapons': {'avail': False, 'files': []}, 'pickups': {'avail': False}}
mpbusiness: {'weapons': {'avail': True, 'files': ['weaponheavypistol.meta', 'weaponspecialcarbine.meta']}, 'pickups': {'avail': True}}
mpvalentines: {'weapons': {'avail': False, 'files': []}, 'pickups': {'avail': False}}
mpvalentines: {'weapons': {'avail': True, 'files': ['weapongusenberg.meta']}, 'pickups': {'avail': True}}
mpbusiness2: {'weapons': {'avail': False, 'files': []}, 'pickups': {'avail': False}}
mpbusiness2: {'weapons': {'avail': True, 'files': ['weaponbullpuprifle.meta']}, 'pickups': {'avail': True}}
mphipster: {'weapons': {'avail': False, 'files

In [5]:
# Crawl data from vanilla files
def update_weapons_meta(old:dict, new:dict):
    # Only update ammo and weapons
    '''
    CWeaponInfoBlob
        Infos
            Item
            <0> AMMOS
                Infos
                    ? Item
            <1> WEAPONS
                Infos
                    ? Item
    '''
    has_new_ammo = (new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos'] is not None)
    if has_new_ammo:
        if isinstance(new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'], list):
            old['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'].extend(
                new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item']
            )
        else:
            old['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'].append(
                new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item']
            )

    has_new_weapon = (new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos'] is not None)
    if has_new_weapon:
        if isinstance(new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'], list):
            old['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'].extend(
                new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item']
            )
        else:
            old['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'].append(
                new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item']
            )

def log_ammo_doc(doc:dict, new:dict, metadata:dict):
    has_new_ammo = (new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos'] is not None)
    if has_new_ammo:
        if isinstance(new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'], list):
            for ammo in new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item']:
                doc[ammo['Name']] = metadata
        else:
            doc[new['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item']['Name']] = metadata

def log_weapon_doc(doc:dict, new:dict, metadata:dict):
    has_new_weapon = (new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos'] is not None)
    if has_new_weapon:
        if isinstance(new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'], list):
            for weapon in new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item']:
                doc[weapon['Name']] = metadata
        else:
            doc[new['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item']['Name']] = metadata

# First round: search in 'base' installment, at root/update/x64/dlcpacks/<dlc>
# Second round: search in 'patch' installment, at root/update/update.rpf/dlc_patch/<dlc>
weapons_meta = ET.parse(source='./vanilla/root/update/update.rpf/common/data/ai/weapons.meta').getroot()
weapons_meta = xmltodict.parse(ET.tostring(weapons_meta))
ammos_doc = {}
weapons_doc = {}
log_ammo_doc(doc=ammos_doc, new=weapons_meta, metadata={'dlc': 'base', 'installment': 'base', 'file': 'weapons.meta', 'path': './vanilla/root/update/update.rpf/common/data/ai/weapons.meta'})
log_weapon_doc(doc=weapons_doc, new=weapons_meta, metadata={'dlc': 'base', 'installment': 'base', 'file': 'weapons.meta', 'path': './vanilla/root/update/update.rpf/common/data/ai/weapons.meta'})

for k_d, v_d in dlclist.items():
    for installment in ['base', 'patch']:
        if v_d[installment]['weapons']['avail']:
            for f in v_d[installment]['weapons']['files']:
                if (k_d == 'base') and installment == 'base':
                    continue
                
                f_xml_path = (
                    f'{VANILLA_PATH}/{v_d['path']}/common/data/ai/{f}' if installment == 'base'
                    else f'{VANILLA_PATH}/update/update.rpf/dlc_patch/{k_d}/common/data/ai/{f}'
                )
                f_xml = ET.parse(source=f_xml_path).getroot()
                f_dict = xmltodict.parse(ET.tostring(f_xml))
                update_weapons_meta(old=weapons_meta, new=f_dict)
                log_ammo_doc(doc=ammos_doc, new=f_dict, metadata={'dlc': k_d, 'installment': installment, 'file': f, 'path': f_xml_path})
                log_weapon_doc(doc=weapons_doc, new=f_dict, metadata={'dlc': k_d, 'installment': installment, 'file': f, 'path': f_xml_path})

In [6]:
if not os.path.isdir(f'{PATH_OUTPUTS}/supplementary-docs'):
    os.makedirs(f'{PATH_OUTPUTS}/supplementary-docs')
    # logger.debug(f"Created folder {PATH_OUTPUTS}.")

with open(f'{PATH_OUTPUTS}/supplementary-docs/ammos_list.csv', 'w') as f:
    f.write('ammo,dlc,installment,file,path\n')
    for key, value in ammos_doc.items():
        f.write(f"{key},{value['dlc']},{value['installment']},{value['file']},{value['path']}\n")

with open(f'{PATH_OUTPUTS}/supplementary-docs/weapons_list.csv', 'w') as f:
    f.write('weapon,dlc,installment,file,path\n')
    for key, value in weapons_doc.items():
        f.write(f"{key},{value['dlc']},{value['installment']},{value['file']},{value['path']}\n")

In [7]:
# Remove the first elements of each duplicated ammos and weapons
# 0 for Ammo, 1 for Weapon, as in CWeaponInfoBlob structure
for i in [0, 1]:
    ELEMENTS_LIST = [e['Name'] for e in list(weapons_meta['CWeaponInfoBlob']['Infos']['Item'][i]['Infos']['Item'])]
    ELEMENTS_DUPLICATES = {e: True for e in ELEMENTS_LIST if ELEMENTS_LIST.count(e) > 1}
    for ii in range(len(weapons_meta['CWeaponInfoBlob']['Infos']['Item'][i]['Infos']['Item'])):
        element = weapons_meta['CWeaponInfoBlob']['Infos']['Item'][i]['Infos']['Item'][ii]
        if ELEMENTS_DUPLICATES.get(element['Name']) is True:
            weapons_meta['CWeaponInfoBlob']['Infos']['Item'][i]['Infos']['Item'][ii] = DELETE_TOKEN
            ELEMENTS_DUPLICATES[element['Name']] = False
    
    pprint(f"{'AMMOS' if i == 0 else 'WEAPONS'} DUPLICATES: {list(ELEMENTS_DUPLICATES.keys())}")
    while DELETE_TOKEN in weapons_meta['CWeaponInfoBlob']['Infos']['Item'][i]['Infos']['Item']:
        weapons_meta['CWeaponInfoBlob']['Infos']['Item'][i]['Infos']['Item'].remove(DELETE_TOKEN)

"AMMOS DUPLICATES: ['AMMO_FLAREGUN', 'AMMO_HOMINGLAUNCHER', 'AMMO_PROXMINE']"
("WEAPONS DUPLICATES: ['WEAPON_FLAREGUN', 'WEAPON_HOMINGLAUNCHER', "
 "'WEAPON_PROXMINE', 'WEAPON_COMBATPDW', 'WEAPON_MARKSMANPISTOL', "
 "'WEAPON_MACHINEPISTOL', 'WEAPON_REVOLVER', 'WEAPON_COMPACTRIFLE', "
 "'WEAPON_HEAVYSNIPER_MK2', 'WEAPON_PISTOL_MK2', 'WEAPON_DOUBLEACTION', "
 "'WEAPON_REVOLVER_MK2']")


In [8]:
def convert_ammo_config_py2xml(input:dict) -> dict:
    out = {}
    for k, v in input.items():
        assert k in ['Name', 'AmmoMax', 'AmmoMax50', 'AmmoMax100']
        
        if k in ['AmmoMax', 'AmmoMax50', 'AmmoMax100']:
            out[k] = {'@value': str(v)}
        else:
            out[k] = v
    return out

weapons_meta_converted = deepcopy(weapons_meta)
DELETE_TOKEN = '!DELETE'
for i_a, ammo in enumerate(list(weapons_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'])):
    for k_convert in list(AMMOS_CONVERSION[ammo['Name']].keys()):
        if k_convert == 'update':
            # If an update is singular, we update it inplace
            # If an update is forking to multiple versions, we add to the end and delete the original
            if isinstance(AMMOS_CONVERSION[ammo['Name']]['update'], list):
                AMMOS_CONVERSION[ammo['Name']]['update'] = AMMOS_CONVERSION[ammo['Name']]['update']
                for u in AMMOS_CONVERSION[ammo['Name']]['update']:
                    template:dict = deepcopy(ammo)
                    template.update(convert_ammo_config_py2xml(u))
                    weapons_meta_converted['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'].append(template)
            elif isinstance(AMMOS_CONVERSION[ammo['Name']]['update'], dict):
                template:dict = deepcopy(ammo)
                template.update(convert_ammo_config_py2xml(AMMOS_CONVERSION[ammo['Name']]['update']))
                weapons_meta_converted['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'][i_a] = template
            else:
                raise TypeError(f'Unknown update type: {type(AMMOS_CONVERSION[ammo["Name"]]["update"])}')
        elif k_convert == 'delete':
            weapons_meta_converted['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'][i_a] = DELETE_TOKEN
        else:
            raise ValueError(f'Unknown conversion type: {k_convert}')
weapons_meta = weapons_meta_converted
while DELETE_TOKEN in weapons_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item']:
    weapons_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item'].remove(DELETE_TOKEN)
print('CONVERTED AMMOS:')
pprint(', '.join(a['Name'] for a in weapons_meta['CWeaponInfoBlob']['Infos']['Item'][0]['Infos']['Item']))

CONVERTED AMMOS:
('AMMO_STUNGUN, AMMO_SNIPER_REMOTE, AMMO_FIREEXTINGUISHER, AMMO_PETROLCAN, '
 'AMMO_MINIGUN, AMMO_GRENADELAUNCHER, AMMO_GRENADELAUNCHER_SMOKE, AMMO_RPG, '
 'AMMO_STINGER, AMMO_GRENADE, AMMO_BALL, AMMO_STICKYBOMB, AMMO_SMOKEGRENADE, '
 'AMMO_BZGAS, AMMO_FLARE, AMMO_MOLOTOV, AMMO_TANK, AMMO_SPACE_ROCKET, '
 'AMMO_PLANE_ROCKET, AMMO_PLAYER_LASER, AMMO_ENEMY_LASER, AMMO_BIRD_CRAP, '
 'AMMO_MG_ARMORPIERCING, AMMO_MG_FMJ, AMMO_MG_INCENDIARY, AMMO_MG_TRACER, '
 'AMMO_PISTOL_FMJ, AMMO_PISTOL_HOLLOWPOINT, AMMO_PISTOL_INCENDIARY, '
 'AMMO_PISTOL_TRACER, AMMO_RIFLE_ARMORPIERCING, AMMO_RIFLE_FMJ, '
 'AMMO_RIFLE_INCENDIARY, AMMO_RIFLE_TRACER, AMMO_SMG_FMJ, '
 'AMMO_SMG_HOLLOWPOINT, AMMO_SMG_INCENDIARY, AMMO_SMG_TRACER, '
 'AMMO_SNIPER_ARMORPIERCING, AMMO_SNIPER_EXPLOSIVE, AMMO_SNIPER_FMJ, '
 'AMMO_SNIPER_INCENDIARY, AMMO_SNIPER_TRACER, AMMO_SHOTGUN_ARMORPIERCING, '
 'AMMO_SHOTGUN_EXPLOSIVE, AMMO_SHOTGUN_HOLLOWPOINT, AMMO_SHOTGUN_INCENDIARY, '
 'AMMO_FIREWORK, AMMO_RAILGUN, AMMO_FLA

In [9]:
def convert_weapon_config_py2xml(input:dict) -> dict:
    out = {}
    for k, v in input.items():
        assert k in ['FireType', 'AmmoInfo', 'Speed']
        
        if k == 'AmmoInfo':
            out[k] = {'@ref': v}
        elif k == 'Speed':
            out[k] = {'@value': str(v)}
        else:
            out[k] = v
    return out

weapons_meta_converted = deepcopy(weapons_meta)
for i_w, weapon in enumerate(list(weapons_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'])):
    for k_convert in list(WEAPONS_CONVERSION[weapon['Name']].keys()):
        if k_convert == 'update':
            template:dict = deepcopy(weapon)
            template.update(convert_weapon_config_py2xml(WEAPONS_CONVERSION[weapon['Name']]['update']))
            weapons_meta_converted['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'][i_w] = template
        elif k_convert == 'metadata':
            pass
        else:
            raise ValueError(f'Unknown conversion type: {k_convert}')
        
weapons_meta = weapons_meta_converted
while DELETE_TOKEN in weapons_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item']:
    weapons_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item'].remove(DELETE_TOKEN)

print('CONVERTED WEAPONS:')
pprint(', '.join(w['Name'] for w in weapons_meta['CWeaponInfoBlob']['Infos']['Item'][1]['Infos']['Item']))

CONVERTED WEAPONS:
('WEAPON_UNARMED, WEAPON_ANIMAL, WEAPON_COUGAR, WEAPON_KNIFE, '
 'WEAPON_NIGHTSTICK, WEAPON_HAMMER, WEAPON_BAT, WEAPON_GOLFCLUB, '
 'WEAPON_CROWBAR, WEAPON_PISTOL, WEAPON_COMBATPISTOL, WEAPON_APPISTOL, '
 'WEAPON_PISTOL50, WEAPON_MICROSMG, WEAPON_SMG, WEAPON_ASSAULTSMG, '
 'WEAPON_ASSAULTRIFLE, WEAPON_CARBINERIFLE, WEAPON_ADVANCEDRIFLE, WEAPON_MG, '
 'WEAPON_COMBATMG, WEAPON_PUMPSHOTGUN, WEAPON_SAWNOFFSHOTGUN, '
 'WEAPON_ASSAULTSHOTGUN, WEAPON_BULLPUPSHOTGUN, WEAPON_STUNGUN, '
 'WEAPON_SNIPERRIFLE, WEAPON_HEAVYSNIPER, WEAPON_REMOTESNIPER, '
 'WEAPON_GRENADELAUNCHER, WEAPON_GRENADELAUNCHER_SMOKE, WEAPON_RPG, '
 'WEAPON_PASSENGER_ROCKET, WEAPON_AIRSTRIKE_ROCKET, WEAPON_STINGER, '
 'WEAPON_MINIGUN, WEAPON_GRENADE, WEAPON_STICKYBOMB, WEAPON_SMOKEGRENADE, '
 'WEAPON_BZGAS, WEAPON_MOLOTOV, WEAPON_FIREEXTINGUISHER, WEAPON_PETROLCAN, '
 'WEAPON_DIGISCANNER, GADGET_NIGHTVISION, GADGET_PARACHUTE, OBJECT, '
 'WEAPON_BRIEFCASE, WEAPON_BRIEFCASE_02, WEAPON_BALL, WEAPON_FLARE, '
 

In [10]:
if not os.path.isdir(PATH_OUTPUTS):
    os.makedirs(PATH_OUTPUTS)
    # logger.debug(f"Created folder {PATH_OUTPUTS}.")
weapons_meta_xml = ET.ElementTree(element=ET.fromstring(text=xmltodict.unparse(
    input_dict=weapons_meta,
    encoding='utf-8',
    pretty=True,
    indent=2,
)))
weapons_meta_xml.write(
    file_or_filename=f'{PATH_OUTPUTS}/weapons.meta',
    encoding="utf-8",
    xml_declaration=True,
)

In [11]:
# Crawl data from vanilla files
# Pickups from pickups.meta
def update_pickups_meta(old:dict, new:dict):
    '''
    CPickupDataManager
        pickupData
            Item
        actionData
        rewardData
    '''
    has_new_pickup = (new['CPickupDataManager']['pickupData']['Item'] is not None)
    if has_new_pickup:
        if isinstance(new['CPickupDataManager']['pickupData']['Item'], list):
            old['CPickupDataManager']['pickupData']['Item'].extend(
                new['CPickupDataManager']['pickupData']['Item']
            )
        else:
            old['CPickupDataManager']['pickupData']['Item'].append(
                new['CPickupDataManager']['pickupData']['Item']
            )

def log_pickup_doc(doc:dict, new:dict, metadata:dict):
    has_new_pickup = (new['CPickupDataManager']['pickupData']['Item'] is not None)
    if has_new_pickup:
        if isinstance(new['CPickupDataManager']['pickupData']['Item'], list):
            for pickup in new['CPickupDataManager']['pickupData']['Item']:
                doc[pickup['Name']] = metadata
        else:
            doc[new['CPickupDataManager']['pickupData']['Item']['Name']] = metadata

pickups_meta = ET.parse(source='./vanilla/root/update/update.rpf/common/data/pickups.meta').getroot()
pickups_meta = xmltodict.parse(ET.tostring(pickups_meta))
pickups_doc = {}
log_pickup_doc(doc=pickups_doc, new=pickups_meta, metadata={'dlc': 'base', 'installment': 'base', 'path': './vanilla/root/update/update.rpf/common/data/pickups.meta'})
# First round: search in 'base' installment, at root/update/x64/dlcpacks/<dlc>
# Second round: search in 'patch' installment, at root/update/update.rpf/dlc_patch/<dlc>
for k_d, v_d in dlclist.items():
    for installment in ['base', 'patch']:
        if v_d[installment]['pickups']['avail']:
            if (k_d == 'base') and (installment == 'base'):
                continue
            
            f_xml_path = (
                f'{VANILLA_PATH}/{v_d['path']}/common/data/pickups.meta' if installment == 'base'
                else f'{VANILLA_PATH}/update/update.rpf/dlc_patch/{k_d}/common/data/pickups.meta'
            )
            f_xml = ET.parse(source=f_xml_path).getroot()
            f_dict = xmltodict.parse(ET.tostring(f_xml))
            update_pickups_meta(old=pickups_meta, new=f_dict)
            log_pickup_doc(doc=pickups_doc, new=f_dict, metadata={'dlc': k_d, 'installment': installment, 'path': f_xml_path})
            

In [12]:
if not os.path.isdir(f'{PATH_OUTPUTS}/supplementary-docs'):
    os.makedirs(f'{PATH_OUTPUTS}/supplementary-docs')
    # logger.debug(f"Created folder {PATH_OUTPUTS}.")

with open(f'{PATH_OUTPUTS}/supplementary-docs/pickups_list.csv', 'w') as f:
    f.write('pickup,dlc,installment,path\n')
    for k, v in pickups_doc.items():
        f.write(f"{k},{v['dlc']},{v['installment']},{v['path']}\n")

In [13]:
# Remove the first elements of each duplicated ammos and weapons
# 0 for Ammo, 1 for Weapon, as in CWeaponInfoBlob structure
PICKUPS_LIST = [p['Name'] for p in list(pickups_meta['CPickupDataManager']['pickupData']['Item'])]
PICKUPS_DUPLICATES = {p: True for p in PICKUPS_LIST if PICKUPS_LIST.count(p) > 1}
for ii in range(len(pickups_meta['CPickupDataManager']['pickupData']['Item'])):
    element = pickups_meta['CPickupDataManager']['pickupData']['Item'][ii]
    if PICKUPS_DUPLICATES.get(element['Name']) is True:
        pickups_meta['CPickupDataManager']['pickupData']['Item'][ii] = DELETE_TOKEN
        PICKUPS_DUPLICATES[element['Name']] = False

pprint(f"PICKUPS DUPLICATES: {list(PICKUPS_DUPLICATES.keys())}")
while DELETE_TOKEN in pickups_meta['CPickupDataManager']['pickupData']['Item']:
    pickups_meta['CPickupDataManager']['pickupData']['Item'].remove(DELETE_TOKEN)

("PICKUPS DUPLICATES: ['PICKUP_WEAPON_PROXMINE', "
 "'PICKUP_WEAPON_HOMINGLAUNCHER', 'PICKUP_AMMO_HOMINGLAUNCHER', "
 "'PICKUP_WEAPON_ASSAULTRIFLE_MK2', 'PICKUP_WEAPON_CARBINERIFLE_MK2', "
 "'PICKUP_WEAPON_COMBATMG_MK2', 'PICKUP_WEAPON_HEAVYSNIPER_MK2', "
 "'PICKUP_WEAPON_PISTOL_MK2', 'PICKUP_WEAPON_SMG_MK2']")


In [14]:
def convert_pickup_config(input:dict) -> dict:
    output = deepcopy(input)
    if output['Name'].startswith('PICKUP_WEAPON_') or output['Name'].startswith('PICKUP_VEHICLE_WEAPON_'):
        for i_r, r in enumerate(output['Rewards']['Item']):
            if r.startswith('REWARD_WEAPON_') or r.startswith('REWARD_AMMO_'):
                output['Rewards']['Item'][i_r] = DELETE_TOKEN
        while DELETE_TOKEN in output['Rewards']['Item']:
            output['Rewards']['Item'].remove(DELETE_TOKEN)
    return output

for i_p, pickup in enumerate(list(pickups_meta['CPickupDataManager']['pickupData']['Item'])):
    pickups_meta['CPickupDataManager']['pickupData']['Item'][i_p] = convert_pickup_config(pickup)

print('CONVERTED PICKUPS:')
pprint(', '.join(p['Name'] for p in pickups_meta['CPickupDataManager']['pickupData']['Item']))

CONVERTED PICKUPS:
('PICKUP_WEAPON_BULLPUPSHOTGUN, PICKUP_WEAPON_ASSAULTSMG, '
 'PICKUP_VEHICLE_WEAPON_ASSAULTSMG, PICKUP_WEAPON_PISTOL50, '
 'PICKUP_VEHICLE_WEAPON_PISTOL50, PICKUP_AMMO_BULLET_MP, '
 'PICKUP_AMMO_MISSILE_MP, PICKUP_AMMO_GRENADELAUNCHER_MP, '
 'PICKUP_WEAPON_ASSAULTRIFLE, PICKUP_WEAPON_CARBINERIFLE, '
 'PICKUP_WEAPON_ADVANCEDRIFLE, PICKUP_WEAPON_MG, PICKUP_WEAPON_COMBATMG, '
 'PICKUP_WEAPON_SNIPERRIFLE, PICKUP_WEAPON_HEAVYSNIPER, '
 'PICKUP_WEAPON_MICROSMG, PICKUP_WEAPON_SMG, PICKUP_ARMOUR_STANDARD, '
 'PICKUP_WEAPON_RPG, PICKUP_WEAPON_MINIGUN, PICKUP_HEALTH_STANDARD, '
 'PICKUP_WEAPON_PUMPSHOTGUN, PICKUP_WEAPON_SAWNOFFSHOTGUN, '
 'PICKUP_WEAPON_ASSAULTSHOTGUN, PICKUP_WEAPON_GRENADE, PICKUP_WEAPON_MOLOTOV, '
 'PICKUP_WEAPON_SMOKEGRENADE, PICKUP_WEAPON_STICKYBOMB, PICKUP_WEAPON_PISTOL, '
 'PICKUP_WEAPON_COMBATPISTOL, PICKUP_WEAPON_APPISTOL, '
 'PICKUP_WEAPON_GRENADELAUNCHER, PICKUP_MONEY_VARIABLE, '
 'PICKUP_GANG_ATTACK_MONEY, PICKUP_WEAPON_STUNGUN, PICKUP_WEAPON_PETROL

In [15]:
if not os.path.isdir(PATH_OUTPUTS):
    os.makedirs(PATH_OUTPUTS)
    # logger.debug(f"Created folder {PATH_OUTPUTS}.")
pickups_meta_xml = ET.ElementTree(element=ET.fromstring(text=xmltodict.unparse(
    input_dict=pickups_meta,
    encoding='utf-8',
    pretty=True,
    indent=2,
)))
pickups_meta_xml.write(
    file_or_filename=f'{PATH_OUTPUTS}/pickups.meta',
    encoding="utf-8",
    xml_declaration=True,
)